# Step1: Act 
#### Background:
**By 2016, Bellabeat, a company that develops wearable products for women, had launched multiple prdoucts, and the products were growing 
a big number of online retailers and to their own e-commerce channel. 
The company had invest in traditional advertisment but put more sources 
on digital markteting on social media and google ads.**
    
    
#### Business task:
   **Gain insights about how do the consumers use the 
   device in order to propse better marketing strategy.**
    
   **Stakeholders:  1.Urška Sršen, co-founder and Chief Creative Officer 
                   2.Sando Mur: Mathematician, executive team
                   3.marketing analytics team**

# Step2: Prepare 

  **- The data is Kaggle:FitBit Fitness Tracker Data (322MB, 18csv files)
  -The data is generated from a survey via Amazon Mechinal Turk between March12 to May12.**
       
#### Is Data ROCCC? Reliable, Original, Comprehensive,Current , Cited
*     Relibable: It only has 30 eligible candidates, it may not be reliable
*     Original: The data is from a third party provider, which makes it less reliable.  
*     Comprehensive: No gender and age, it is hard to define the comprehensiveness
*     Current : It's the data 5 years ago, not current. 
*     Cited : Yes
         
#### Data Selection: 
   **dailyActivity_merged.csv , dailyCalories_merged.csv, sleepDay.csv, weightLoginfo_me**

# Step3: Process (data cleaning)


In [ ]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import os

In [ ]:
daily_activity = pd.read_csv('../input/fitbit/Fitabase Data 4.12.16-5.12.16/dailyActivity_merged.csv')
sleep_day = pd.read_csv('../input/fitbit/Fitabase Data 4.12.16-5.12.16/sleepDay_merged.csv')

In [ ]:
daily_activity.head()

In [ ]:
daily_activity.info() # No missing value, total 15columns

In [ ]:
daily_activity['ActivityDate']= pd.to_datetime(daily_activity['ActivityDate'])
daily_activity['day_of_week'] = daily_activity['ActivityDate'].dt.strftime('%A')
day_of_week = daily_activity['day_of_week']
daily_activity.drop(labels=['day_of_week'], axis=1, inplace = True)
daily_activity.insert(2, 'day_of_week', day_of_week)
daily_activity
# Reconstruct the datetime to pandas to_datetime, and add new column(day_of_week)

# Step4: Analyze

In [ ]:
daily_activity.describe()

In [ ]:
sleep_day.head()

In [ ]:
sleep_day.info() # There's no missing value, Total 5 columns

In [ ]:
sleep_day['SleepDay']= pd.to_datetime(sleep_day['SleepDay'])
sleep_day.head()
# Reconstruct the datetime in sleepday

In [ ]:
sleep_day['Id'].nunique()
#check the unique id in sleep_day

# Step5: Share 
    visualization and finding

In [ ]:
df = sleep_day.groupby('Id').mean().reset_index()
def sleep_time(x):
    if x<360:
        return "less than 6 hours"
    elif x>=360 and x<480:
        return "6-8 hours"
    else:
        return "more than 8 hours"
#Sleep_time function to set up a range of 3 categories
df['TotalMinutesAsleep_ranges'] = df['TotalMinutesAsleep'].apply(lambda x: sleep_time(x))
sns.countplot(x="TotalMinutesAsleep_ranges", data=df, order=["less than 6 hours", "6-8 hours", "more than 8 hours"]);
plt.title('Sleep time ranges');
# We want the actual time a person asleep

In [ ]:
df1 = daily_activity.groupby('Id').mean().reset_index()
merged_df = pd.merge(df, df1, on="Id")
def active_minutes(x):
    if x<20:
        return "less than 20 min"
    elif x>20:
        return "more than 20 min"
# 2 categories, we use the number close to the mean of VeryActiveMinutes(21.16)
merged_df['VeryActiveMinutes_ranges'] = merged_df['VeryActiveMinutes'].apply(lambda x: active_minutes(x))    
sns.set(font_scale=1.0)
sns.countplot(x='TotalMinutesAsleep_ranges', data=merged_df, hue='VeryActiveMinutes_ranges'
         , order=['less than 6 hours', '6-8 hours', 'more than 8 hours'])
plt.title('Relationship between VeryActiveMinutes and sleep time');
# Here we want to check the relationship between sleep time and VeryActiveMinute

We can see here from the graph, Relationship between Very_Active_Minutes and sleep_time,
most of the people who sleep less than 6 hours' active minutes tend to be less than 20 minutes.
People who sleep in the range of 6-8 hours' active minutes seem the same.


****So the more a person sleep does not show it will have higher VeryActiveMinutes.****

In [ ]:
sns.heatmap(daily_activity[['TotalSteps','TrackerDistance','VeryActiveMinutes','FairlyActiveMinutes',
               'LightlyActiveMinutes','SedentaryMinutes','Calories']].corr()
            ,cmap='Wistia',linecolor='white',linewidths=2,annot=True)
plt.title('Correlation of daily_activity')

We can see from the graph, there are better correlation between TrackerDistance and VeryActiveMinutes.(0.68)

The Calories and VeryActiveMinutes have better correlation than others.(0.65)

In [ ]:
Very_Active_Min = daily_activity["VeryActiveMinutes"].mean()
Fairly_Active_Min = daily_activity["FairlyActiveMinutes"].mean()
Lightly_Active_Min = daily_activity["LightlyActiveMinutes"].mean()
Sedentary_Min = daily_activity["SedentaryMinutes"].mean()
size=[Very_Active_Min, Fairly_Active_Min, Lightly_Active_Min , Sedentary_Min]
labels = ["VeryActiveMinutes","FairlyActiveMinutes", "LightlyActiveMinutes", "SedentaryMinutes"]
seperate = [0, 0, 0.1, 0.1]
plt.style.use("seaborn-pastel")
plt.pie(size, labels = labels, 
        explode = seperate, autopct = "%1.1f%%")
plt.title("Usage segemtation")
plt.show()

From the graph we can see, 81.3% of the users stay in sedentaryminutes.

In [ ]:
plt.style.use("ggplot")
plt.figure(figsize=(7,4)) # specify size of the chart

n, bins, patches = plt.hist(daily_activity.day_of_week, 50,width=0.5, alpha=0.75)

plt.xlabel("Day of the week")
plt.ylabel("Frequency")
plt.title("Number of times users logged in week")
plt.show()

Most of the users logged in the weakdays; especailly from Tuesday to Thursday, not on the weekend.

# Step6: Act

* Summary of findings:  
 1.  Users do not logged in their acitivty.
 2.  Users who fall asleep around 6-8 hours are more actively using.
 3.  Very active users have better correlation with calories.
 4.  81.3% of the users track for Sedentary activities.
 5.  Users tend to track the activities during weekdays.


* Recommendation:  
 1. Promote the products which can automated the data to Fitbit, so user can see                             their analysis, and based on the analysis suggest some personal trainining for user.
 2. From the daily_activity data we can see, some of the data did not be stored, it seems like their some technical issues which can be improved.
 3. Having some indoor or outdoor personal training on weekdays which can be done with family or alone.
 4. Goal setting, and having the tracker to remind the users for following up on the progress.